# Day 16

https://adventofcode.com/2022/day/16

Wasn't able to figure this one out on my own. So I followed (and understood) this solution by _hyper-neutrino_:

https://www.youtube.com/watch?v=bLMj50cpOug

https://github.com/hyper-neutrino/advent-of-code/blob/main/2022/day16p1.py


## Relevant algorithms

https://en.wikipedia.org/wiki/Depth-first_search

https://en.wikipedia.org/wiki/Breadth-first_search

https://en.wikipedia.org/wiki/Floyd%E2%80%93Warshall_algorithm

## Part 1

In [1]:
test_data = """\
Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II"""
test_data

'Valve AA has flow rate=0; tunnels lead to valves DD, II, BB\nValve BB has flow rate=13; tunnels lead to valves CC, AA\nValve CC has flow rate=2; tunnels lead to valves DD, BB\nValve DD has flow rate=20; tunnels lead to valves CC, AA, EE\nValve EE has flow rate=3; tunnels lead to valves FF, DD\nValve FF has flow rate=0; tunnels lead to valves EE, GG\nValve GG has flow rate=0; tunnels lead to valves FF, HH\nValve HH has flow rate=22; tunnel leads to valve GG\nValve II has flow rate=0; tunnels lead to valves AA, JJ\nValve JJ has flow rate=21; tunnel leads to valve II'

In [2]:
# Parse input data
from collections import deque

def parse_data(data):
    # flow rate of valves
    flow_rate = {}
    # targets from valve
    targets = {}

    for line in data.split("\n"):
        line = line.strip()
        valve = line.split()[1]
        flow = int(line.split(";")[0].split("=")[1])
        target = line.split("to ")[1].split(" ", 1)[1].split(", ")
        flow_rate[valve] = flow
        targets[valve] = target
    return flow_rate, targets

flow_rate, targets = parse_data(test_data)
print("flow_rate:", flow_rate)
print("targets", targets)

flow_rate: {'AA': 0, 'BB': 13, 'CC': 2, 'DD': 20, 'EE': 3, 'FF': 0, 'GG': 0, 'HH': 22, 'II': 0, 'JJ': 21}
targets {'AA': ['DD', 'II', 'BB'], 'BB': ['CC', 'AA'], 'CC': ['DD', 'BB'], 'DD': ['CC', 'AA', 'EE'], 'EE': ['FF', 'DD'], 'FF': ['EE', 'GG'], 'GG': ['FF', 'HH'], 'HH': ['GG'], 'II': ['AA', 'JJ'], 'JJ': ['II']}


In [3]:
def build_dist_map(flow_rate, targets, verbose=False):
    # Calculate (shortest) distances from each valve to every other valve (with nonzero flow rate)
    dists = {}
    nonempty = []

    for valve in flow_rate:
        #verbose = True
        if verbose: print(f"valve {valve}")

        # skip empty if not start valve AA or flow rate = 0
        if valve != "AA" and not flow_rate[valve]:
            if verbose: print(f"\tskipping {valve} with flow rate {flow_rate[valve]}")
            continue

        # list valves that have flow rate > 0
        if valve != "AA":
            if verbose: print(f"\tadd {valve} with flow rate {flow_rate[valve]} to non empty list")
            nonempty.append(valve)


        dists[valve] = {valve: 0, "AA": 0}

        # set of visited valves
        visited = {valve}

        # init queue with distance = 0 and position = current valve
        queue = deque([(0, valve)]) # (distance, position)

        if verbose: print("\tCalculating distances")

        # breadth first search
        while queue:
            distance, position = queue.popleft()
            #if verbose: print(f"\t\tqueue item: {(distance, position)}, neighbors: {targets[position]}")
            for neighbor in targets[position]:
                # skip if already visited (would not be shortest distance)
                if neighbor in visited:
                    continue
                visited.add(neighbor)

                # if flow rate > 0 add distance to map with distance so far + 1
                if flow_rate[neighbor]:
                    dists[valve][neighbor] = distance + 1
                # add valve to queue
                queue.append((distance + 1, neighbor))
            #if verbose: print(f"\t\tvisited: {visited}")

        del dists[valve][valve]
        if valve != "AA":
            del dists[valve]["AA"]
        #if verbose: print("\t", dists)
        
    # turn list of nonempty lists into dict
    indices = {}

    for index, element in enumerate(nonempty):
        indices[element] = index

    return dists, indices

dists, indices = build_dist_map(flow_rate, targets, verbose=True)
print("\nDists:")
print(dists)

valve AA
	Calculating distances
valve BB
	add BB with flow rate 13 to non empty list
	Calculating distances
valve CC
	add CC with flow rate 2 to non empty list
	Calculating distances
valve DD
	add DD with flow rate 20 to non empty list
	Calculating distances
valve EE
	add EE with flow rate 3 to non empty list
	Calculating distances
valve FF
	skipping FF with flow rate 0
valve GG
	skipping GG with flow rate 0
valve HH
	add HH with flow rate 22 to non empty list
	Calculating distances
valve II
	skipping II with flow rate 0
valve JJ
	add JJ with flow rate 21 to non empty list
	Calculating distances

Dists:
{'AA': {'DD': 1, 'BB': 1, 'CC': 2, 'EE': 2, 'JJ': 2, 'HH': 5}, 'BB': {'CC': 1, 'DD': 2, 'EE': 3, 'JJ': 3, 'HH': 6}, 'CC': {'DD': 1, 'BB': 1, 'EE': 2, 'JJ': 4, 'HH': 5}, 'DD': {'CC': 1, 'EE': 1, 'BB': 2, 'JJ': 3, 'HH': 4}, 'EE': {'DD': 1, 'CC': 2, 'HH': 3, 'BB': 3, 'JJ': 4}, 'HH': {'EE': 3, 'DD': 4, 'CC': 5, 'BB': 6, 'JJ': 7}, 'JJ': {'DD': 3, 'BB': 3, 'CC': 4, 'EE': 4, 'HH': 7}}


In [4]:
def solution1(data, time=30, valve="AA"):
    flow_rate, targets = parse_data(data)
    dists, indices = build_dist_map(flow_rate, targets)
    
    cache = {}
    # (recursive) depth first search
    # use bitmask to track which valves are already open
    def dfs(time, valve, bitmask):
        if (time, valve, bitmask) in cache:
            return cache[(time, valve, bitmask)]

        # maximum released water (pressure + time)
        maxval = 0
        for neighbor in dists[valve]:
            # set bit at position of valve to 1
            bit = 1 << indices[neighbor]        
            # skip if already open
            if bitmask & bit:
                continue
            # else open (uses 1 minute), use dist map to get time to get there
            remtime = time - dists[valve][neighbor] - 1

            # stop if no remaining time 
            if remtime <= 0:
                continue
            # recursively search all child nodes
            maxval = max(maxval, dfs(remtime, neighbor, bitmask | bit) + flow_rate[neighbor] * remtime)

        cache[(time, valve, bitmask)] = maxval
        return maxval
    
    sol = dfs(time, valve, 0)
    print("Solution:", sol)
    return sol

assert solution1(test_data) == 1651
print("test passed")

Solution: 1651
test passed


In [5]:
with open("day16.txt") as f:
    inp_data = f.read()

sol1 = solution1(inp_data)
assert sol1 == 2250
print("test passed")

Solution: 2250
test passed


## Part 2

In [6]:
def solution2(data, time=26, valve="AA"):
    flow_rate, targets = parse_data(data)
    dists, indices = build_dist_map(flow_rate, targets)
    
    cache = {}
    # (recursive) depth first search
    # use bitmask to track which valves are already open
    def dfs(time, valve, bitmask):
        if (time, valve, bitmask) in cache:
            return cache[(time, valve, bitmask)]

        # maximum released water (pressure + time)
        maxval = 0
        for neighbor in dists[valve]:
            # set bit at position of valve to 1
            bit = 1 << indices[neighbor]        
            # skip if already open
            if bitmask & bit:
                continue
            # else open (uses 1 minute), use dist map to get time to get there
            remtime = time - dists[valve][neighbor] - 1

            # stop if no remaining time 
            if remtime <= 0:
                continue
            # recursively search all child nodes
            maxval = max(maxval, dfs(remtime, neighbor, bitmask | bit) + flow_rate[neighbor] * remtime)

        cache[(time, valve, bitmask)] = maxval
        return maxval
    
    # maximum number of valve states (0b1111111...)
    b = (1 << len(indices)) - 1

    sol = 0

    for i in range((b + 1) // 2):
        sol = max(sol, dfs(time, valve, i) + dfs(time, valve, b ^ i))

    print("Solution:", sol)
    return sol


sol2 = solution2(test_data)

assert sol2 == 1707
print("test passed")

Solution: 1707
test passed


In [7]:
import time
_t = time.time()
sol2 = solution2(inp_data)
print("elapsed time:", time.time() - _t)
assert sol2 == 3015
print("test passed")

Solution: 3015
elapsed time: 40.82982277870178
test passed
